In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import numba as nb
import scipy.interpolate as spip
from time import process_time

from interpolate import cubic_spline, CubicSpline
from new_interp import spline_numba
from cinterpolate import CubicSpline as spline_cython

/Users/deyanmihaylov/Documents/Work/python310/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from cinterpolate import method1

In [20]:
N = 4000
X = np.random.uniform(size=N)
X.sort()
Y = np.random.uniform(size=N)

# X = np.array([0., 1., 2., 3., 4.])
# Y = np.array([21., 24., 24., 18., 16.])

a = np.zeros(N)
b = np.zeros(N)
c = np.zeros(N)
d = np.zeros(N)


In [21]:
%timeit method1(X, Y, a, b, c, d)


24.5 s ± 1.55 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
method1(X, Y, a, b, c, d)

print(d)

[ 21.          22.17857143 -15.53571429 110.78571429]


In [13]:
sp_sp = spip.CubicSpline(X, Y, bc_type="natural")

sa, sb, sc, sd = sp_sp.c

In [17]:
x = 2.5

sa[2] * (x-2)**3 + sb[2] * (x-2)**2 + sc[2] * (x - 2) + sd[2]

21.127232142857142

In [6]:
x = 2.5

a[2] * x**3 + b[2] * x**2 + c[2] * x + d[2]

21.127232142857164

In [3]:
def max_rel_diff(x, y):
    return np.max(np.abs((x - y) / x))

def max_abs_diff(x, y):
    return np.max(np.abs(x - y))

In [18]:
N = 4
X = np.random.uniform(size=N)
X.sort()
Y = np.random.uniform(size=N)

scipy_spline = spip.CubicSpline(X, Y, bc_type="natural")
numba_spline_old = CubicSpline(X, Y)
# a, b, c, d, numba_spline_new = spline_numba(X, Y)
cython_spline = spline_cython(X, Y)

# %timeit scipy_spline = spip.CubicSpline(X, Y, bc_type="natural")
# %timeit numba_spline_old = CubicSpline(X, Y)
# %timeit numba_spline_new = spline_numba(X, Y)
# %timeit spline_cython(X, Y)

scipy_d, scipy_c, scipy_b, scipy_a = scipy_spline.c

numba_a = numba_spline_old.a
numba_b = numba_spline_old.b
numba_c = numba_spline_old.c
numba_d = numba_spline_old.d

cython_a = cython_spline.a
cython_b = cython_spline.b
cython_c = cython_spline.c
cython_d = cython_spline.d
# print(max_abs_diff(a, np.array(cython_spline.a)))
# print(max_abs_diff(b, np.array(cython_spline.b)))
# print(max_abs_diff(c, np.array(cython_spline.c)))
# print(max_abs_diff(d, np.array(cython_spline.d)))


In [19]:
print(max_abs_diff(scipy_a, numba_a))
print(max_abs_diff(scipy_b, numba_b))
print(max_abs_diff(scipy_c, numba_c))
print(max_abs_diff(scipy_d, numba_d))

0.0
1.4488698797559414
1.3322676295501878e-15
5.329070518200751e-15


In [16]:
from cinterpolate import tri_diag_solve
from new_interp import tri_diag_solve as tdsnb

In [19]:
N = 100000000 
f1 = np.random.uniform(size=N)
f2 = np.random.uniform(size=N)
f3 = np.random.uniform(size=N)
f4 = np.random.uniform(size=N)

max_rel_diff(tri_diag_solve(f1, f2, f3, f4), tdsnb(f1, f2, f3, f4))

0.0

In [8]:
z = np.random.uniform(low=X.min(), high=X.max())
w = np.array([z])

%timeit scipy_spline(z)
%timeit numba_spline_old.eval(z)
%timeit numba_spline_new(w)
%timeit cython_spline(w)


4.5 µs ± 70.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
2.7 µs ± 32.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
3.23 µs ± 250 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
15.3 µs ± 113 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [11]:
test_array = np.random.uniform(low=X.min(), high=X.max(), size=100)

rel_diff = (scipy_spline(test_array) - cython_spline(test_array)) / scipy_spline(test_array)
np.max(np.abs(rel_diff))


0.002568664176321869

In [12]:
%timeit scipy_spline(test_array)
%timeit cython_spline(test_array)

8.24 µs ± 328 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
37.9 µs ± 2.85 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [16]:
np.array(cython_spline.b)

array([-390.18816831,   89.49955701,   13.87803287,  -51.37324011,
        -48.50873385,   17.01519089,  -11.75360343,   -9.24302563,
          7.78560907,    3.11868167,   -9.68744385,   25.11615695,
        -42.00549626,  -47.42376289,  106.44586247, -306.7063883 ,
        241.12078653,   40.41153019, -529.84235886,  223.64052591,
       -140.43678171,  141.60512694, -296.7066927 , -384.92846661,
        128.97161665,  -40.57557286,  -18.14729272,   26.87898103,
        -23.25835722,   10.60151416,   23.30922972,  -55.75710561,
        -70.53698622,   61.15513973])

In [14]:
Y

array([0.78045858, 0.17788129, 0.36413759, 0.8992841 , 0.84526243,
       0.20856463, 0.28241681, 0.40900933, 0.28936396, 0.67663465,
       0.81049792, 0.9359569 , 0.06079716, 0.63763808, 0.4984415 ,
       0.6598545 , 0.72367369, 0.48009277, 0.83566315, 0.07713948,
       0.90681428, 0.7396249 , 0.50390262, 0.91379776, 0.38756481,
       0.08320292, 0.96791747, 0.10231251, 0.66746931, 0.6864403 ,
       0.07419727, 0.11515538, 0.9316228 , 0.28327665, 0.86859014])

In [1]:
from cinterpolate import spline_params
import numpy as np
import numba as nb

In [4]:
N = 100
X = np.random.uniform(size=N)
X.sort()
Y = np.random.uniform(size=N)

a = np.empty(N)
b = np.empty(N)
c = np.empty(N)
d = np.empty(N)


In [6]:
%timeit spline_params(a, b, c, d, X, Y)

1.46 µs ± 34.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [7]:
@nb.njit()
def calc_spline_params(x, y):
    n = x.size
    a = y

    return a

In [9]:
%timeit calc_spline_params(X, Y)

640 ns ± 5.33 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [1]:
import numba as nb
import numpy as np

/Users/deyanmihaylov/Documents/Work/python310/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
@nb.njit(cache=True, inline = 'always')
def TDMA(a, b, c, d):
    n = len(d)
    w = np.zeros(n-1, float)
    g = np.zeros(n, float)
    p = np.zeros(n, float)
    
    w[0] = c[0]/b[0]
    g[0] = d[0]/b[0]

    for i in range(1,n-1):
        w[i] = c[i]/(b[i] - a[i-1]*w[i-1])
    for i in range(1,n):
        g[i] = (d[i] - a[i-1]*g[i-1])/(b[i] - a[i-1]*w[i-1])
    p[n-1] = g[n-1]
    for i in range(n-1,0,-1):
        p[i-1] = g[i-1] - w[i-1]*p[i]
    return p

def TDMAnew(a, b, c, x):
    n = x.size

    c[0] = c[0] / b[0]
    x[0] = x[0] / b[0]

    for i in range(1, n):
        if i < n-1:
            c[i] = c[i] / (b[i] - a[i-1] * c[i-1])
        
        x[i] = (x[i] - a[i-1] * x[i-1]) / (b[i] - a[i-1] * c[i-1])

    for i in range(n-2, -1, -1):
        x[i] -= c[i] * x[i + 1]

def method2(x, y):
    n = x.size
    mu = np.zeros(n-1)
    ell = np.zeros(n-1)
    d = np.zeros(n)

    h = np.diff(x)

    for i in range(0, n-2):
        mu[i] = h[i] / (h[i] + h[i+1])
        ell[i+1] = h[i+1] / (h[i] + h[i+1])
        d[i+1] = 6 * (
            (y[i+2] - y[i+1]) / (h[i+1] * (x[i+2] - x[i]))
            - (y[i+1] - y[i]) / (h[i] * (x[i+2] - x[i]))
        )

    diag = 2 * np.ones(n)

    # ell[0] = 2 * (2 * h[0] + h[1]) / (h[0] - h[1])
    # mu[n-2] = 2 * (2 * h[n-2] + h[n-3]) / (h[n-2] - h[n-3])

    # d[0] = 12 * (
    #     ((y[2] - y[1]) * h[0] / h[1] - y[1] + y[0]) / (h[0] * h[0] - h[1] * h[1])
    # )
    # d[n-1] = 12 * (y[n-1] - y[n-2] - (y[n-2] - y[n-3]) * h[n-2] / h[n-3]) / (h[n-2] * h[n-2] - h[n-3] * h[n-3])

    mu[0] = 0
    diag[1] = 2 + h[0] / h[1]
    ell[1] = 1 - h[0] / h[1]

    mu[n-3] = 1 - h[-1] / h[-2]
    ell[n-2] = 0
    diag[n-2] = 2 + h[-1] / h[-2]
    
    print(mu)
    print(ell)
    print(d)
    M = TDMA(mu[1:-1], diag[1:-1], ell[1:-1], d[1:-1])
    print(M)
    M0 = ((h[0] + h[1]) * M[0] - h[0] * M[1]) / h[1]
    Mn = ((h[-2] + h[-1]) * M[-1] - h[-1] * M[-2]) / h[-2]

    M = np.concatenate(([M0], M, [Mn]))

    a = np.diff(M) / (6 * h)
    b = M[:-1] / 2
    c = np.diff(y) / h - M[1:] * h / 6 - M[:-1] * h / 3
    d = y[:-1]

    return a, b, c, d

def solve_nearly_tridiagonal_old(
    a, c, x, ur, ll,
):
    n = x.size
    r = np.zeros(n)
    c[0] = c[0] / 2
    r[0] = ur / 2
    x[0] = x[0] / 2
    
    for i in range(1, n-2):
        c[i] = c[i] / (2 - a[i-1] * c[i-1])
        r[i] = - a[i-1] * r[i-1] / (2 - a[i-1] * c[i-1])
        x[i] = (x[i] - a[i-1] * x[i-1]) / (2 - a[i-1] * c[i-1])

    r[n-2] = (c[n-2] - a[n-3] * r[n-3]) / (2 - a[n-3] * c[n-3])
    x[n-2] = (x[n-2] - a[n-3] * x[n-3]) / (2 - a[n-3] * c[n-3])
    
    for i in range(n-3, -1, -1):
        r[i] -= c[i] * r[i+1]
        x[i] -= c[i] * x[i+1]

    x[n-1] = (x[n-1] - a[n-2] * x[n-2] - ll * x[0]) / (2 - a[n-2] * r[n-2] - ll * r[0])

    for i in range(n-2, -1, -1):
        x[i] -= r[i] * x[n-1]

    return x

def solve_nearly_tridiagonal(
    a, c, x
):
    n = x.size

    a[0] = a[0] / 2
    c[0] = c[0] / 2
    x[0] = x[0] / 2
    
    for i in range(1, n-2):
        c[i] = c[i] / (2 - a[i] * c[i-1])
        x[i] = (x[i] - a[i] * x[i-1]) / (2 - a[i] * c[i-1])
        a[i] = - a[i] * a[i-1] / (2 - a[i] * c[i-1])

    x[n-2] = (x[n-2] - a[n-2] * x[n-3]) / (2 - a[n-2] * c[n-3])
    a[n-2] = (c[n-2] - a[n-2] * a[n-3]) / (2 - a[n-2] * c[n-3])
    
    for i in range(n-3, -1, -1):
        x[i] -= c[i] * x[i+1]
        a[i] -= c[i] * a[i+1]

    x[n-1] = (x[n-1] - a[n-1] * x[n-2] - c[n-1] * x[0]) / (2 - a[n-1] * a[n-2] - c[n-1] * a[0])

    for i in range(n-2, -1, -1):
        x[i] -= a[i] * x[n-1]

    return x

def period(x, y):
    n = x.size
    mu = np.zeros(n-1)
    ell = np.zeros(n-1)
    d = np.zeros(n)

    h = np.diff(x)

    for i in range(0, n-2):
        mu[i] = h[i] / (h[i] + h[i+1])
        ell[i+1] = h[i+1] / (h[i] + h[i+1])
        d[i+1] = 6 * (
            (y[i+2] - y[i+1]) / (h[i+1] * (x[i+2] - x[i]))
            - (y[i+1] - y[i]) / (h[i] * (x[i+2] - x[i]))
        )

    diag = 2 * np.ones(n)
    A = np.zeros((n, n))

    for i in range(n):
        A[i, i] = diag[i]
        if i > 0:
            A[i, i-1] = mu[i-1]
        if i < n-1:
            A[i, i+1] = ell[i]

    A[0, 1] = h[0] / (h[0] + h[-1])
    A[0, n-2] = h[-1] / (h[0] + h[-1])
    A[n-2, 0] = h[-1] / (h[-2] + h[-1])

    ell[0] = h[0] / (h[0] + h[-1])
    ur = h[-1] / (h[0] + h[-1])
    ll = h[-1] / (h[-2] + h[-1])
    
    d[0] = 6 * (
        (y[1] - y[0]) / (h[0] * (h[0] + h[-1]))
        - (y[-1] - y[-2]) / (h[-1] * (h[0] + h[-1]))
    )
    P = A[:-1, :-1]
    q = d[:-1]
    M = np.linalg.solve(A[:-1, :-1], d[:-1])
    print(M)
    M = np.append(M, M[0])

    return P, q
    a = np.diff(M) / (6 * h)
    b = M[:-1] / 2
    c = np.diff(y) / h - M[1:] * h / 6 - M[:-1] * h / 3
    d = y[:-1]

    return a, b, c, d

def period_new(x, y):
    n = x.size
    mu = np.zeros(n-1)
    ell = np.zeros(n-1)
    d = np.zeros(n-1)

    h = np.diff(x)

    for i in range(0, n-2):
        mu[i+1] = h[i] / (h[i] + h[i+1])
        ell[i+1] = h[i+1] / (h[i] + h[i+1])
        d[i+1] = 6 * (
            (y[i+2] - y[i+1]) / (h[i+1] * (x[i+2] - x[i]))
            - (y[i+1] - y[i]) / (h[i] * (x[i+2] - x[i]))
        )

    ell[0] = h[0] / (h[0] + h[n-2])
    mu[0] = h[n-2] / (h[0] + h[n-2])
    ell[n-2] = h[n-2] / (h[n-3] + h[n-2])
    
    d[0] = 6 * (
        (y[1] - y[0]) / (h[0] * (h[0] + h[n-2]))
        - (y[n-1] - y[n-2]) / (h[n-2] * (h[0] + h[n-2]))
    )
    
    M = solve_nearly_tridiagonal(mu, ell, d)
    M = np.append(M, M[0])

    a = np.diff(M) / (6 * h)
    b = M[:-1] / 2
    c = np.diff(y) / h - M[1:] * h / 6 - M[:-1] * h / 3
    d = y[:-1]

    return a, b, c, d

In [3]:
X = np.array([0., 1., 2., 3.])
Y = np.array([0., 0.5, 2., 0.])

In [4]:
import scipy.interpolate as spip

# sp_spline = spip.CubicSpline(X, Y, bc_type=((1, 0.2), (1, -1)))
# sp_spline = spip.CubicSpline(X, Y, bc_type=((2, -0.3), (2, 3.3)))
sp_spline = spip.CubicSpline(X, Y, bc_type="periodic")

In [5]:
sp_spline.c

array([[-0.5, -1.5,  2. ],
       [ 2.5,  1. , -3.5],
       [-1.5,  2. , -0.5],
       [ 0. ,  0.5,  2. ]])

In [6]:
from cinterpolate import CubicSpline as cs

In [7]:
c_cs = cs(X, Y, 'periodic')
print(np.array(c_cs.a))
print(np.array(c_cs.b))
print(np.array(c_cs.c))
print(np.array(c_cs.d))


[-0.5 -1.5  2. ]
[ 2.5  1.  -3.5]
[-1.5  2.  -0.5]
[0.  0.5 2. ]


In [8]:
np.allclose(sp_spline.c, np.vstack((c_cs.a,c_cs.b,c_cs.c,c_cs.d)))

True

In [16]:
%timeit -n 100000 cs(X, Y, 'not-a-knot')

4.63 µs ± 896 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [17]:
%timeit sp_spline = spip.CubicSpline(X, Y, bc_type="not-a-knot")

127 µs ± 3.02 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [74]:
N = 10

a = np.random.uniform(size=N-1)
b = 2 * np.ones(N)
c = np.random.uniform(size=N-1)
d = np.random.uniform(size=N-1)

In [75]:
y = TDMA(a, b, c, d)
TDMAnew(a, b, c, d)
np.allclose(d, y)

True

In [76]:
TDMA(a, b, c, d)

array([ 0.11147846,  0.03772752,  0.18435356, -0.02627814, -0.01205115,
        0.07057727,  0.06056884,  0.18243457, -0.06709042])

In [77]:
TDMAnew(a, b, c, d)
d

array([ 0.11147846,  0.03772752,  0.18435356, -0.02627814, -0.01205115,
        0.07057727,  0.06056884,  0.18243457, -0.06709042])

In [14]:
vec = np.random.uniform(size=100)

In [15]:
%timeit vec1 = vec[:-1]

170 ns ± 13.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
